In [2]:
import pandas as pd 

In [3]:
df=pd.read_csv("NikeProductDescriptions (1).csv")
df.head()

,Title,Subtitle,Product Description
0,Nike Air Force 1 '07,Men's Shoes,It doesn't get more legendary than this. Desig...
1,Nike Air Max Dawn SE,Men's Shoes,Find out what moves you with the Air Max Dawn....
2,Nike SB Dunk Low Pro Premium,Skate Shoes,Pack your style—on your feet. Bringing a fresh...
3,Nike Air Force 1 Mid '07 LX,Men's Shoes,The celebrations just keep coming. Unbox the A...
4,Nike Air Force 1 Mid '07,Men's Shoes,"Got your fave colour yet? No worries, the Colo..."


In [4]:
subtitles_to_keep = ["Men's Shoes", "Men's T-Shirt", "Women's Shoes", "Skate Shoes", "Older Kids' T-Shirt"]
filtered_df = df[df['Subtitle'].isin(subtitles_to_keep) | df['Subtitle'].str.contains("Shorts", case=False)]
filtered_df.loc[filtered_df['Subtitle'].str.contains("Shorts", case=False), 'Subtitle'] = "Shorts"

C:\Users\admin-27619\AppData\Local\Temp\ipykernel_10360\3376879734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[filtered_df['Subtitle'].str.contains("Shorts", case=False), 'Subtitle'] = "Shorts"


In [5]:
sampled = filtered_df[["Product Description"]].dropna().sample(n=5, random_state=42).reset_index(drop=True)
sampled

,Product Description
0,"Let classic, easy-to-wear AF-1 style rise to t..."
1,There's no pedalling required to enjoy these s...
2,"Rebuilt using insights from Grant Taylor, the ..."
3,"When it came time to build his signature shoe,..."
4,Gear up for training. Play tag with your BFFs....


In [6]:
for i, desc in enumerate(sampled["Product Description"]):
    print(f"{i}. {desc}\n")

0. Let classic, easy-to-wear AF-1 style rise to the occasion with the Nike Air Force 1 PLT.AF.ORM. Its elegantly shaped and lifted midsole delivers a proud, new voice to the hoops franchise. The leather on the upper breaks in easily and ages to soft perfection while the sculpted collar and pillowy heel keep it comfy. Captivate your audience.

1. There's no pedalling required to enjoy these snug, stretchy shorts. Like your favourite leggings (and made from the same fabric as them), they're a perfect fit for warmer weather.

2. Rebuilt using insights from Grant Taylor, the Nike SB Zoom Blazer Low Pro GT is a fresh take on a favourite skate shoe. The updated design has higher taping to give you more durability.

3. When it came time to build his signature shoe, Ishod Wair was all in from start to finish. Infused with elements taken from the iconic hoops shoes of the '90s, (did you know that basketball was Ishod's 1st love?) and built with all the durability you need to skate hard—seriousl

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(sampled["Product Description"])
#simularité cosinus
cosine_sim_matrix = cosine_similarity(X_tfidf)

In [10]:
cosine_df = pd.DataFrame(cosine_sim_matrix, 
                         index=[f"Text {i}" for i in range(5)],
                         columns=[f"Text {i}" for i in range(5)])
cosine_df

,Text 0,Text 1,Text 2,Text 3,Text 4
Text 0,1.000000,0.118018,0.126474,0.254499,0.131974
Text 1,0.118018,1.000000,0.068609,0.090671,0.186769
Text 2,0.126474,0.068609,1.000000,0.191757,0.131979
Text 3,0.254499,0.090671,0.191757,1.000000,0.157470
Text 4,0.131974,0.186769,0.131979,0.157470,1.000000


In [12]:
texts = sampled["Product Description"].tolist()
# matrice 5 x 5
jaccard_matrix = pd.DataFrame(0.0, index=[f"Text {i}" for i in range(5)],
                                    columns=[f"Text {i}" for i in range(5)])
for i in range(5):
    for j in range(5):
        set_i = set(texts[i].lower().split())
        set_j = set(texts[j].lower().split())
        intersection = set_i.intersection(set_j)
        union = set_i.union(set_j)
        score = len(intersection) / len(union)
        jaccard_matrix.iloc[i, j] = score
jaccard_matrix


,Text 0,Text 1,Text 2,Text 3,Text 4
Text 0,1.000000,0.054795,0.066667,0.108696,0.076087
Text 1,0.054795,1.000000,0.089286,0.037500,0.095890
Text 2,0.066667,0.089286,1.000000,0.088608,0.092105
Text 3,0.108696,0.037500,0.088608,1.000000,0.050000
Text 4,0.076087,0.095890,0.092105,0.050000,1.000000


In [13]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()


C:\Users\admin-27619\anaconda3\envs\nlp_bert_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\admin-27619\anaconda3\envs\nlp_bert_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [16]:
import numpy as np
from tqdm import tqdm
def get_bert_embedding(text):
    # Tokenisation de la phrase
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    
    # Désactiver le gradient 
    with torch.no_grad():
        outputs = model(**inputs)
    
    # On extrait le vecteur [CLS] comme représentation globale
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding
embeddings = np.array([get_bert_embedding(desc) for desc in tqdm(sampled["Product Description"])])


100%|██████████| 5/5 [00:01<00:00,  2.98it/s]


In [17]:
embeddings[0][:10]  # Les 10 premières valeurs du 1er embedding

array([ 0.01908397, -0.33997953,  0.12956531, -0.64775836, -0.5130369 ,
       -0.51012695,  0.30023336,  0.6959071 ,  0.41129142, -0.75964725],
      dtype=float32)

Analyse des résultats
Avec la similarité cosinus appliquée à la matrice TF-IDF, on observe des scores modérés entre certains textes, notamment ceux traitant de chaussures de skate ou de sport. Cette méthode capte efficacement les recoupements lexicaux et met en évidence les descriptions qui partagent des termes techniques ou commerciaux fréquents. Par exemple, les textes contenant des mots comme "durabilité", "chaussure", ou "Nike SB" obtiennent des scores de similarité plus élevés entre eux. Cependant, cette approche reste sensible à la variation de vocabulaire, même pour des produits similaires.

La similarité de Jaccard, plus stricte, donne des scores globalement faibles. Cela s’explique par le fait que cette méthode ne considère que la proportion de mots strictement identiques entre deux textes. Or, les descriptions marketing utilisent souvent des formulations variées pour parler de produits proches. Même deux textes décrivant des shorts ou des chaussures auront peu de mots en commun au sens strict, ce qui réduit leur score, malgré une proximité sémantique.

En revanche, avec la méthode BERT basée sur les embeddings sémantiques, les résultats apparaissent plus cohérents du point de vue du sens. Cette approche capture les similarités même lorsque les mots diffèrent, ce qui permet de reconnaître comme proches des textes qui parlent du même type de produit ou qui remplissent une même fonction (par exemple : confort, performance, sport). BERT parvient à regrouper des textes qui seraient faiblement liés selon TF-IDF ou Jaccard, ce qui en fait une méthode particulièrement pertinente pour des tâches où la signification prime sur la forme.